In [1]:
import json
import requests
from datetime import datetime, timedelta

In [2]:
!git clone https://github.com/Rohan-J123/PyroPredictorModel

Cloning into 'PyroPredictorModel'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 111 (delta 84), reused 100 (delta 76), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 8.46 MiB | 32.31 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [3]:
!pip install earthengine-api geemap

import ee
import geemap
import json
import requests
from datetime import datetime, timedelta
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='wildfireprediction-442613')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
def vegetationtypeNdviEvi(LATITUDE, LONGITUDE, START_DATE, END_DATE):
    lat, lon = LATITUDE, LONGITUDE
    point = ee.Geometry.Point([lon, lat])

    def sample_indices(image):
        sampled_point = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=500
        )
        return ee.Feature(None, {
            'date': image.date().format('YYYY-MM-dd'),
            'NDVI': sampled_point.get('NDVI'),
            'EVI': sampled_point.get('EVI')
        })

    dataset = ee.ImageCollection('MODIS/061/MOD13A1') \
                .filter(ee.Filter.date(START_DATE, END_DATE))

    indices = dataset.select(['NDVI', 'EVI'])

    sampled = indices.map(sample_indices)

    sampled_fc = ee.FeatureCollection(sampled)
    sampled_list = sampled_fc.getInfo()

    ndvi_value = None
    evi_value = None
    if len(sampled_list['features']) > 0:
        date = sampled_list['features'][0]['properties']['date']
        ndvi_value = sampled_list['features'][0]['properties']['NDVI']
        evi_value = sampled_list['features'][0]['properties']['EVI']
        print(f"Date: {date}, NDVI: {ndvi_value}, EVI: {evi_value}")
    if ndvi_value and evi_value:
        return [ndvi_value, evi_value]
    else:
        return [None, None]

In [ ]:
def vegetationtypeLaiFpar(LATITUDE, LONGITUDE, START_DATE, END_DATE):
    lat, lon = LATITUDE, LONGITUDE
    point = ee.Geometry.Point([lon, lat])

    def sample_indices(image):
        sampled_point = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=500
        )
        return ee.Feature(None, {
            'date': image.date().format('YYYY-MM-dd'),
            'LAI': sampled_point.get('Lai'),
            'FPAR': sampled_point.get('Fpar')
        })

    dataset = ee.ImageCollection('MODIS/061/MCD15A3H') \
                .filter(ee.Filter.date(START_DATE, END_DATE))

    indices = dataset.select(['Lai', 'Fpar'])

    sampled = indices.map(sample_indices)

    sampled_fc = ee.FeatureCollection(sampled)
    sampled_list = sampled_fc.getInfo()

    lai_value = None
    fpar_value = None
    if len(sampled_list['features']) > 0:
        date = sampled_list['features'][0]['properties']['date']
        lai_value = sampled_list['features'][0]['properties']['LAI']
        fpar_value = sampled_list['features'][0]['properties']['FPAR']
        print(f"Date: {date}, LAI: {lai_value}, FPAR: {fpar_value}")
    if lai_value and fpar_value:
        return [lai_value, fpar_value]
    else:
        return [None, None]

In [6]:
output_dir = "/content/Fire_Dataset_Vegetation"
os.makedirs(output_dir, exist_ok=True)

In [9]:
file_paths = ["/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_1.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_2.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_3.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_4.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_5.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_6.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_7.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_8.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_9.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Combined/fire_batch_10.geojson"]

In [13]:
i = 1
for file_path in file_paths:
    with open(file_path, 'r') as file:
        geo_data = json.load(file)
        output_data = []
        for feature in geo_data['features']:
            [LONGITUDE, LATITUDE] = feature['geometry']['coordinates']
            CURR_DATE = feature['properties']['fromdate']

            date_obj = datetime.strptime(CURR_DATE, "%Y-%m-%d")
            fifteen_days_behind = date_obj - timedelta(days=15)
            START_DATE_15 = fifteen_days_behind.strftime("%Y-%m-%d")
            six_days_behind = date_obj - timedelta(days=6)
            START_DATE_6 = fifteen_days_behind.strftime("%Y-%m-%d")

            date_obj = datetime.strptime(CURR_DATE, "%Y-%m-%d")
            fifteen_days_forward = date_obj + timedelta(days=15)
            END_DATE_15 = fifteen_days_forward.strftime("%Y-%m-%d")
            size_days_forward = date_obj + timedelta(days=6)
            END_DATE_6 = fifteen_days_forward.strftime("%Y-%m-%d")

            [NDVI, evi] = vegetationtypeNdviEvi(LATITUDE, LONGITUDE, START_DATE_15, END_DATE_15)
            [lai, fpar] = vegetationtypeLaiFpar(LATITUDE, LONGITUDE, START_DATE_6, END_DATE_6)

            feature["vegetation_indices"] = {"NDVI": NDVI, "EVI": evi, "LAI": lai, "FPAR": fpar}
            output_data.append(feature)
        geo_data['features'] = output_data
    with open(f"/content/Fire_Dataset_Vegetation/fire_batch_{i}.geojson", 'w') as out_file:
          json.dump(geo_data, out_file, indent=4)
    i += 1

Streaming output truncated to the last 5000 lines.
Date: 2022-03-18, LAI: 6, FPAR: 27
Date: 2022-03-22, LAI: 8, FPAR: 31
Date: 2022-03-26, LAI: 7, FPAR: 29
Date: 2022-03-30, LAI: 6, FPAR: 25
Date: 2022-03-22, NDVI: 4297, EVI: 2525
Date: 2022-04-07, NDVI: 4665, EVI: 3004
Date: 2022-03-10, LAI: 4, FPAR: 22
Date: 2022-03-14, LAI: 4, FPAR: 23
Date: 2022-03-18, LAI: 5, FPAR: 25
Date: 2022-03-22, LAI: 5, FPAR: 26
Date: 2022-03-26, LAI: 5, FPAR: 27
Date: 2022-03-30, LAI: 6, FPAR: 31
Date: 2022-03-22, NDVI: 663, EVI: 323
Date: 2022-04-07, NDVI: None, EVI: None
Date: 2022-03-14, LAI: None, FPAR: None
Date: 2022-03-18, LAI: None, FPAR: None
Date: 2022-03-22, LAI: None, FPAR: None
Date: 2022-03-26, LAI: None, FPAR: None
Date: 2022-03-30, LAI: None, FPAR: None
Date: 2022-03-22, NDVI: 2611, EVI: 1599
Date: 2022-04-07, NDVI: 2981, EVI: 1740
Date: 2022-03-14, LAI: 4, FPAR: 20
Date: 2022-03-18, LAI: 4, FPAR: 20
Date: 2022-03-22, LAI: 4, FPAR: 20
Date: 2022-03-26, LAI: 4, FPAR: 21
Date: 2022-03-30, LAI

In [14]:
!zip -r Fire_Dataset_Vegetation.zip Fire_Dataset_Vegetation

  adding: Fire_Dataset_Vegetation/ (stored 0%)
  adding: Fire_Dataset_Vegetation/fire_batch_3.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_8.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_5.geojson (deflated 92%)
  adding: Fire_Dataset_Vegetation/fire_batch_2.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_7.geojson (deflated 92%)
  adding: Fire_Dataset_Vegetation/fire_batch_1.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_9.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_10.geojson (deflated 91%)
  adding: Fire_Dataset_Vegetation/fire_batch_4.geojson (deflated 92%)
  adding: Fire_Dataset_Vegetation/fire_batch_6.geojson (deflated 91%)
